#### Information Retrieval :: Recommendations with Implicit Feedbacks


# Recommendations with Implicit Feedbacks: Rating Prediction and Top-K Item Recommendation



*Goals of this tutorial:* Understand matrix factorization (MF) using explicit feedback and Bayesian Personalized Ranking (BPR) using implicit feedback for recommendation. Explore different methods for two real-world recommendation senarios: rating prediction and top-K item recommendation.

# Part 1. Matrix Factorization for Rating Prediction 

In some platforms, such as MovieLens, users express their preference on items using explict feedback like ratings.

In this part, a matrix factorization will be implemented to predict ratings on MovieLens data. After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

## Part 1a: Load the Data

In the Movie dataset, there are about 65,000 ratings in total. We split the rating data into two sets. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [0]:
# load the data, then print out the number of
# movies and users in each of train and test sets.
# Your Code Here...
import pandas as pd
import numpy as np

train_df = pd.read_csv("train_movie.csv", sep=',', header=None, skiprows=1)
train_df.columns = ['user', 'movie', 'rating']
test_df = pd.read_csv('test_movie.csv', sep=',', header=None, skiprows=1)
test_df.columns = ['user', 'movie', 'rating']

print ("Number of unique users in train set is ", len(train_df.user.unique()) )
print ("Number of unique movie in train set is ", len(train_df.movie.unique()) )
print ("Number of unique users in test set is ", len(test_df.user.unique()) )
print ("Number of unique movie in test set is ", len(test_df.movie.unique()) )
print (train_df)

Number of unique users in train set is  541
Number of unique movie in train set is  1211
Number of unique users in test set is  541
Number of unique movie in test set is  1211
       user  movie  rating
0         0      4       4
1         0     23       5
2         0     25       5
3         0     31       3
4         0     33       5
...     ...    ...     ...
45277   540   1202       3
45278   540   1204       3
45279   540   1205       4
45280   540   1206       4
45281   540   1207       4

[45282 rows x 3 columns]


## Part 1b: Matrix Factorization

In this part, we will implement a matrix factorization mechanism for recommendations. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model.

The metrics MAE and RMSE of implemented MF model will be shown on test set.

In [0]:
# Cosine Similarity of User–User Collaborative Filtering
import math
from functools import reduce

rating_matrix = train_df.pivot_table(index='user', columns='movie', values='rating').fillna(0).to_numpy()
actual_rating = test_df.pivot_table(index='user', columns='movie', values='rating').fillna(0).to_numpy()


In [0]:
import numpy as np
# The class of MF
class MF:
    def __init__(self, train, test, hidden=40, epoch=10, lr=0.01, bias=False):
        self.bias = bias
        self.train = train
        self.test = test
        self.mask = 1.0 * (train > 0)
        self.testmask = 1.0 * (test > 0)
        self.num_train = np.sum(self.mask)
        self.num_test  = np.sum(self.testmask)
        self.num_user, self.num_item = train.shape
        self.hidden = hidden

        if self.bias:
            self.global_bias = np.sum(train) / np.sum(self.mask)
            self.user_bias = np.sum((train - self.global_bias) * self.mask, axis=1) / np.sum(self.mask, axis=1)
            self.item_bias = np.sum((train - self.global_bias) * self.mask, axis=0) / np.sum(self.mask, axis=0)
        else:
            self.user_bias = np.zeros(self.num_user)
            self.item_bias = np.zeros(self.num_item)
            self.global_bias = 0.0

        self.epoch = epoch

        self.sample_row, self.sample_col = self.train.nonzero()
        self.n_samples = len(self.sample_row)
        self.lr = lr


    def fit(self):
        # initialize
        self.P = np.random.random((self.num_user, self.hidden))
        self.Q = np.random.random((self.num_item, self.hidden))

        for itr in range(self.epoch):
            self.training_indices = np.arange(self.n_samples)
            np.random.shuffle(self.training_indices)
            for idx in self.training_indices:
                u = self.sample_row[idx]
                i = self.sample_col[idx]
                rating = self.train[u, i]
                e = rating - self.P[u, :].dot(self.Q[i, :].T) - self.global_bias - self.item_bias[i] - self.user_bias[u]
                self.P[u, :] += self.lr * (e * self.Q[i, :])
                self.Q[i, :] += self.lr * (e * self.P[u, :])

            Rec = self.P.dot(self.Q.T) + self.global_bias + self.item_bias + self.user_bias.reshape((-1, 1))
            error = np.sqrt(np.sum(np.square(Rec * self.mask - self.train)) / self.num_train)
            print('iters={0}, RMSE={1}'.format(itr, error))
        
        # Calculate the RMSE, MAE between predicted and trained
        self.Rec_mf = (self.P.dot(self.Q.T) + self.global_bias + self.item_bias + self.user_bias.reshape((-1, 1)))
        RMSE = np.sqrt(np.sum(np.square(self.Rec_mf * self.testmask - self.test)) / self.num_test)
        MAE = np.sum(np.abs(self.Rec_mf * self.testmask - self.test)) / self.num_test
        return self.P, self.Q, self.Rec_mf, RMSE, MAE



In [0]:
# train an MF model without bias, print the RMSE for each epoch
mf = MF(rating_matrix, actual_rating, hidden=40, epoch=20, lr=0.01, bias=False)
P_mf, Q_mf, Rec_mf, RMSE, MAE = mf.fit()

iters=0, RMSE=1.0180046342552176
iters=1, RMSE=0.8711010328503331
iters=2, RMSE=0.7989419214059273
iters=3, RMSE=0.7573126422744588
iters=4, RMSE=0.7260011624308345
iters=5, RMSE=0.6970840164352864
iters=6, RMSE=0.672424358607756
iters=7, RMSE=0.6529578252431153
iters=8, RMSE=0.6284094327191848
iters=9, RMSE=0.6085078032776038
iters=10, RMSE=0.5881611040385031
iters=11, RMSE=0.5699243416449129
iters=12, RMSE=0.5537492815058511
iters=13, RMSE=0.5367899067026205
iters=14, RMSE=0.5213113521035709
iters=15, RMSE=0.5056982253345237
iters=16, RMSE=0.4916547901656696
iters=17, RMSE=0.4786446660081976
iters=18, RMSE=0.46677792052145994
iters=19, RMSE=0.45314279244986727


In [0]:
# P_mf, Q_mf, Rec_mf = mf.fit()
print("RMSE =", str(RMSE))
print("MAE =", str(MAE))

RMSE = 1.020489876333291
MAE = 0.7957720435290563


Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

 

I find **P** and **Q** by using gradient descent. AThe biggest advantage of gradient descent is that it can be fast on computations, because it can be done on matrix calculations on each iteration.
<br/>The disadvantage of gradient descent is that it takes lots of iterations to converge when the learning rate is not set properly. Also, it requires a large memory to store Intermediate parameters. Furthermore, if the matrix is sparse, this procedure may not be powerful.

## Part 1c: Improve MF

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system or considering a neighborhood like this paper [Factorization Meets the Neighborhood: a Multifaceted Collaborative Filtering Model](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf). *You can do anything you like to improve MAE and RMSE.*

In [0]:
import numpy as np
# The class of MF
class ImproveMF:
    def __init__(self, train, test, hidden=40, epoch=10, lr=0.01, bias=False, lambda1 = 0.02, lambda2 = 0.03):
        self.bias = bias
        self.train = train
        self.test = test
        self.mask = 1.0 * (train > 0)
        self.testmask = 1.0 * (test > 0)
        self.num_train = np.sum(self.mask)
        self.num_test  = np.sum(self.testmask)
        self.num_user, self.num_item = train.shape
        self.hidden = hidden
        self.lambda1 = lambda1
        self.lambda2 = lambda2

        if self.bias:
            self.global_bias = np.sum(train) / np.sum(self.mask)
            self.user_bias = np.sum((train - self.global_bias) * self.mask, axis=1) / np.sum(self.mask, axis=1)
            self.item_bias = np.sum((train - self.global_bias) * self.mask, axis=0) / np.sum(self.mask, axis=0)
        else:
            self.user_bias = np.zeros(self.num_user)
            self.item_bias = np.zeros(self.num_item)
            self.global_bias = 0.0

        self.epoch = epoch

        self.sample_row, self.sample_col = self.train.nonzero()
        self.n_samples = len(self.sample_row)
        self.lr = lr


    def fit(self):
        # initialize
        self.P = np.random.random((self.num_user, self.hidden))
        self.Q = np.random.random((self.num_item, self.hidden))
        
        for itr in range(self.epoch):
            self.training_indices = np.arange(self.n_samples)
            np.random.shuffle(self.training_indices)
            for idx in self.training_indices:
                u = self.sample_row[idx]
                i = self.sample_col[idx]
                rating = self.train[u, i]
                e = rating - self.P[u, :].dot(self.Q[i, :].T) - self.global_bias - self.item_bias[i] - self.user_bias[u]
                e = e + self.lambda1 * pow(self.P[u, :], 2) + self.lambda2 * (pow(self.Q[i, :].T, 2) + pow(self.item_bias[i], 2) + pow(self.user_bias[u], 2))
                self.P[u, :] += self.lr * (e * self.Q[i, :])
                self.Q[i, :] += self.lr * (e * self.P[u, :])

            Rec = self.P.dot(self.Q.T) + self.global_bias + self.item_bias + self.user_bias.reshape((-1, 1))
            error = np.sqrt(np.sum(np.square(Rec * self.mask - self.train)) / self.num_train)
            print('iters={0}, RMSE={1}'.format(itr, error))
        
        # Calculate the RMSE, MAE between predicted and trained
        self.Rec_mf = (self.P.dot(self.Q.T) + self.global_bias + self.item_bias + self.user_bias.reshape((-1, 1)))
        RMSE = np.sqrt(np.sum(np.square(self.Rec_mf * self.testmask - self.test)) / self.num_test)
        MAE = np.sum(np.abs(self.Rec_mf * self.testmask - self.test)) / self.num_test
        return self.P, self.Q, self.Rec_mf, RMSE, MAE


In [0]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test
# train an MF model without bias, print the RMSE for each epoch
mf = ImproveMF(rating_matrix, actual_rating, hidden=3, epoch=30, lr=0.002, bias=False, lambda1=0.02, lambda2=0.03)
P_mf, Q_mf, Rec_mf, RMSE, MAE = mf.fit()

iters=0, RMSE=2.0523402425812525
iters=1, RMSE=1.5148829460013733
iters=2, RMSE=1.2644762249585697
iters=3, RMSE=1.1219333004882837
iters=4, RMSE=1.0308865750839322
iters=5, RMSE=0.9698184970130348
iters=6, RMSE=0.9277808498585163
iters=7, RMSE=0.8978907544551247
iters=8, RMSE=0.8765143164823072
iters=9, RMSE=0.8605543230385659
iters=10, RMSE=0.8486017248564154
iters=11, RMSE=0.8395262857172707
iters=12, RMSE=0.8324515484463253
iters=13, RMSE=0.8270979450143148
iters=14, RMSE=0.8227810965941166
iters=15, RMSE=0.819735293295987
iters=16, RMSE=0.8168638438765574
iters=17, RMSE=0.8146697740013251
iters=18, RMSE=0.8127362800426613
iters=19, RMSE=0.8112916281627744
iters=20, RMSE=0.8101892981434299
iters=21, RMSE=0.8089792956188272
iters=22, RMSE=0.8080264557633803
iters=23, RMSE=0.8071712839285646
iters=24, RMSE=0.8064694312358457
iters=25, RMSE=0.8058901614135167
iters=26, RMSE=0.8051688408493616
iters=27, RMSE=0.804685528976083
iters=28, RMSE=0.8040220852254698
iters=29, RMSE=0.803867439

In [0]:
# P_mf, Q_mf, Rec_mf = mf.fit()
print("RMSE =", str(RMSE))
print("MAE =", str(MAE))

RMSE = 0.8490170234446313
MAE = 0.6607409045407778


Please explain what you do to improve the recommendation in 1-2 paragraphs.

Compared with Part1b, I introduced regularization to avoid overfitting by adding different lambda parameters for ***P*** and ***Q***.

# Part 2. Bayesian Personalized Ranking (BPR) for Top-K Item Recommendation

Compared with rating prediction in part 1, a more popular scenario recently is personalized top-K item ranking for each user based on the user's implicit feedback. Examples include ranking videos on YouTube and ranking products on Aamzon. In practice, users tend to provide implicit feedback (e.g., the user clicked a product URL on Amazon or played a video on YouTube) rather than explicit feedback (e.g., ratings or reviews) in most cases.

In this part, you will experiment with Bayesian Personalized Ranking (BPR) to rank items on a [Spotify Playlist Recommendation Dataset](http://people.tamu.edu/~yunhe/pubs/AttListCIKM2019.pdf). If a user ever followed a playlist, this interaction is treated as an implicit feedback. In our sampled dataset, there are ~10,000 users and ~7,000 playlists.

BPR can generate scores of items for each user. You should rank all items based on the scores for each user and evaluate the ranking performance.

For example, if user 0 has two interacted playlists 23, 78 in test.txt. If the top-10 playlists for user 0 returned by BPR is [12,45,78,34,23,90,134,33,46,9], then the precision@10 for user 0 is 0.2 because the two playlists in test.txt are recommended in top-10: 2/10=0.2. Please report NDCG@10 in this part.

## Load the Data

Please download the dataset from Piazza. There are about 90,000 interactions in total, which are split into training.txt, validation.txt and text.txt. You will train on train.txt, tune hyperparameters on validation.txt and report final result on test.txt in terms of NDCG@10. 

Each of the train and test files has lines having this format: UserID, PlaylistID, 1.0. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [0]:
# load the data, then print out the number of
# playlists and users in each of train and test sets.
# Your Code Here...
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.txt', sep="\t", header=None)
train_df.columns = ["userid", "playlistid", "one"]
validation_df = pd.read_csv('validation.txt', sep="\t", header=None)
validation_df.columns = ["userid", "playlistid", "one"]
test_df = pd.read_csv('validation.txt', sep="\t", header=None)
test_df.columns = ["userid", "playlistid", "one"]

print ("Number of unique users in train set is ", len(train_df.userid.unique()) )
print ("Number of unique movie in train set is ", len(train_df.playlistid.unique()) )
print ("Number of unique users in validate set is ", len(validation_df.userid.unique()) )
print ("Number of unique movie in validate set is ", len(validation_df.playlistid.unique()) )
print ("Number of unique users in test set is ", len(test_df.userid.unique()) )
print ("Number of unique movie in test set is ", len(test_df.playlistid.unique()) )

num_users = len(train_df.userid.unique())
num_items = len(train_df.playlistid.unique())

Number of unique users in train set is  10183
Number of unique movie in train set is  7787
Number of unique users in validate set is  5895
Number of unique movie in validate set is  3674
Number of unique users in test set is  5895
Number of unique movie in test set is  3674


## BPR by Using Package

Compared with MF, BPR is more complicated to implement. In this part, you can use a BPR package to experiment with top-K item recommendation. Some good packages include https://github.com/benfred/implicit.

In [0]:
#!pip3 install -Iv implicit==0.4.0
#!pip3 uninstall implicit
!pip install lightfm

     |████████████████████████████████| 307kB 3.4MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707643 sha256=f3325809d9ef48efe7440f4dcf2033ef79f37226573c9a80dee456be37e36412
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
## your code to call other BPR packages for top-K recommendation.
# Report average NDCG@10 for all users on test.txt
import numpy as np
import scipy.sparse as sp
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
import pandas as pd
import math

def GenerateUserItemDict(TrainFile):
    fp = open(TrainFile)
    lines = fp.readlines()
    useritemTrainDict = {}
    for line in lines:
        lineStr = line.strip().split('\t')
        userId = int(lineStr[0])
        itemId = int(lineStr[1])
        if userId not in useritemTrainDict:
            useritemTrainDict[userId] = [itemId]
        else:
            useritemTrainDict[userId].append(itemId)
    fp.close()
    train = sp.dok_matrix((num_users, num_items), dtype=np.int8)
    for user_id, movie_ids in useritemTrainDict.items():
        train[user_id, movie_ids] = 1
    train = train.tocsr() # (num_users, num_items) sparse matrix 
    return useritemTrainDict, train

def Model_Predict(train, test, useritemTestDict, learningrate, no_component):
    model = LightFM(learning_rate=learningrate, loss='bpr', no_components=no_component)
    model.fit(train, epochs=30)
    #train_precision = precision_at_k(model, train, k=10).mean()
    #test_precision  = precision_at_k(model, test, k=10).mean()
    train_auc = auc_score(model, train).mean()
    test_auc = auc_score(model, test).mean()
    #print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
    print('AUC: train %.2f, validate %.2f.' % (train_auc, test_auc))

    ## Calculate NDCG@10
    n_users, n_items = test.shape
    ndcg10 = 0.0
    for user_id in useritemTestDict:
        scores = model.predict(user_id, np.arange(n_items))
        top10_items = np.argsort(-scores)[:10]

        dcgPerUser = 0
        idcgPerUser = 0
        rank = 0
        groundtruthLength = len(useritemTestDict[user_id])
        if groundtruthLength >= 10:
            groundtruthLength = 10
        for index in range(groundtruthLength):
            rank = rank + 1
            idcgPerUser += (1 / float(math.log(rank + 1, 2)))
        rank = 0
        for itemIndex in top10_items:
            if itemIndex in useritemTestDict[user_id]:
                position = rank + 1
                dcgPerUser += 1 / float(math.log(position + 1, 2))
            rank = rank + 1
        ndcgPerUser = float(dcgPerUser) / float(idcgPerUser)
        ndcg10 = ndcg10 + ndcgPerUser
    print('NDCG@10: validate' + str(float(ndcg10)/float(len(useritemTestDict))))
    return model, float(ndcg10)/float(n_users)

useritemTrainDict, train = GenerateUserItemDict("train.txt")
useritemValiDict, validate = GenerateUserItemDict("validation.txt")
print ("Model1")
model1, ndcg1 = Model_Predict (train, validate, useritemValiDict, 0.05, 10)
print ("Model2")
model2, ndcg2 = Model_Predict (train, validate, useritemValiDict, 0.05, 50)
print ("Model3")
model3, ndcg3 = Model_Predict (train, validate, useritemValiDict, 0.05, 40)
print ("Model4")
model4, ndcg4 = Model_Predict (train, validate, useritemValiDict, 0.05, 45)
print ("Model5")
model5, ndcg5 = Model_Predict (train, validate, useritemValiDict, 0.05, 35)


Model1
AUC: train 0.92, validate 0.71.
NDCG@10: validate0.06587136507731556
Model2
AUC: train 0.95, validate 0.72.
NDCG@10: validate0.07501440439036912
Model3
AUC: train 0.95, validate 0.72.
NDCG@10: validate0.0765644704173029
Model4
AUC: train 0.95, validate 0.72.
NDCG@10: validate0.07505338460275243
Model5
AUC: train 0.95, validate 0.72.
NDCG@10: validate0.07517939236564417


In [0]:
#### Test #### (Using model3 since it has the best NDCG@10 on validate.txt)
useritemTestDict, test = GenerateUserItemDict("test.txt")
## Calculate NDCG@10
n_users, n_items = test.shape
ndcg10 = 0.0
for user_id in useritemTestDict:
    scores = model3.predict(user_id, np.arange(n_items))
    top10_items = np.argsort(-scores)[:10]

    dcgPerUser = 0
    idcgPerUser = 0
    rank = 0
    if len(useritemTestDict[user_id]) !=0:
        groundtruthLength = len(useritemTestDict[user_id])
        if groundtruthLength >= 10:
            groundtruthLength = 10
        for index in range(groundtruthLength):
            rank = rank + 1
            idcgPerUser += 1 / float(math.log(rank + 1, 2))
        rank = 0
        for itemIndex in top10_items:
            if itemIndex in useritemTestDict[user_id]:
                position = rank + 1
                dcgPerUser += 1 / float(math.log(position + 1, 2))
            rank = rank + 1
        ndcgPerUser = float(dcgPerUser) / float(idcgPerUser)
        ndcg10 = ndcg10 + ndcgPerUser
print('NDCG@10: test ' + str(float(ndcg10)/float(len(useritemTestDict))))

test_auc = auc_score(model3, test).mean()
print('AUC: test ' + str(test_auc))

NDCG@10: test 0.07449423713979982
AUC: test 0.71989447


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

In [0]:
#!pip install LightFM
#https://making.lyst.com/lightfm/docs/lightfm.html
#https://github.com/lyst/lightfm/blob/master/examples/quickstart/quickstart.ipynb
# Using the lightfm package to implement the BPR